In [0]:
# fully from scratch

In [0]:
import tensorflow as tf


In [0]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units = 32, input_dim = 32):
        super(Linear, self).__init__()

        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value = w_init(shape = (input_dim, units), dtype = 'float32'), trainable = True)

        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value = b_init(shape = (units, ), dtype = 'float32'), trainable = True)

    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.ones((2, 2))

#__init__ method called
linear = Linear(4, 2)

#call method called
y = linear(x)

print(y)

tf.Tensor(
[[-0.19652408 -0.0966994  -0.04165873  0.07429425]
 [-0.19652408 -0.0966994  -0.04165873  0.07429425]], shape=(2, 4), dtype=float32)


In [0]:
assert linear.weights == [linear.w, linear.b]


In [0]:
# using add_weight method

In [0]:
class Linear(tf.keras.layers.Layer):

    def __init__(self, units = 32, input_dim = 32):
        super(Linear, self).__init__()

        self.w = self.add_weight(shape = (input_dim, units), initializer = 'random_normal', trainable = True)

        self.b = self.add_weight(shape = (units, ), initializer = 'zeros')

    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

x = tf.ones([2, 2])
linear = Linear(4, 2)
y = linear(x)

        

In [0]:
# non-trainable weights cannot taken durig backpropogation

In [0]:
# it is recommend to initialize weights in build function

In [0]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units = 32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape = (input_shape[-1], self.units), initializer = 'random_normal', trainable = True)
        self.b = self.add_weight(shape = (self.units, ), initializer = 'random_normal', trainable = True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b



In [0]:
linear_layer = Linear(32)
y = linear_layer(x)

In [0]:
y

<tf.Tensor: shape=(2, 32), dtype=float32, numpy=
array([[ 0.15316638, -0.00408052, -0.03939756, -0.03556833,  0.11247923,
        -0.1363092 , -0.08122693,  0.15696126,  0.16924721, -0.07014477,
        -0.0465714 , -0.03588439,  0.11284909,  0.01192444, -0.02960798,
        -0.00604918,  0.0540213 , -0.09195482, -0.04905231, -0.01379287,
         0.01985393, -0.20700842, -0.10167944, -0.08707403,  0.13204019,
        -0.0651741 , -0.0039934 , -0.00381964,  0.00098265, -0.07091521,
         0.01455342, -0.03218016],
       [ 0.15316638, -0.00408052, -0.03939756, -0.03556833,  0.11247923,
        -0.1363092 , -0.08122693,  0.15696126,  0.16924721, -0.07014477,
        -0.0465714 , -0.03588439,  0.11284909,  0.01192444, -0.02960798,
        -0.00604918,  0.0540213 , -0.09195482, -0.04905231, -0.01379287,
         0.01985393, -0.20700842, -0.10167944, -0.08707403,  0.13204019,
        -0.0651741 , -0.0039934 , -0.00381964,  0.00098265, -0.07091521,
         0.01455342, -0.03218016]], dtyp

In [0]:
#layers canbe recursively callable 

# we can assign a layer as an attribute of the anotehr layer, the outer layer will keep tracking oof the weights of the inner layer

In [0]:
class MLPBlock(tf.keras.layers.Layer):
    
    def __init__(self):
        super(MLPBlock, self).__init__()

        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    
    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)



mlp = MLPBlock()
y = mlp(tf.ones((3, 64)))

In [0]:
print(len(mlp.weights))
print(len(mlp.trainable_weights))

6
6


In [0]:
# losses can also be recursively collected during the forward pass

In [0]:
class ActivityRegularisationLayer(tf.keras.layers.Layer):
    def __init__(self, rate = 1e-2):
        super(ActivityRegularisationLayer, self).__init__()
        self.rate = rate

    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

In [0]:
class OuterLayer(tf.keras.layers.Layer):

    def __init__(self):
        super(OuterLayer, self).__init__()
        self.activity_reg = ActivityRegularisationLayer(1e-2)

    def call(self, inputs):
        return self.activity_reg(inputs)

layer = OuterLayer()
assert len(layer.losses) == 0

_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1